In [1]:
import xarray as xr
import s3fs
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
from datetime import datetime
import numpy as np

from dask.diagnostics import ProgressBar

# from dask.distributed import Client, LocalCluster

In [2]:
s3 = s3fs.S3FileSystem(key="K1CQ7M1DMTLUFK182APD", secret="3JuZAQm5I03jtpijCpHOdkAsJDNLNfZxBpM15Pi0", client_kwargs=dict(endpoint_url="https://rgw.met.no"))

## Dask with gateway

In [13]:
from dask.distributed import Client
from dask_gateway import Gateway
gateway = Gateway()

In [14]:
cluster = gateway.new_cluster(worker_memory=4, worker_cores=1)
cluster.scale(4)
client = Client(cluster) # create a dask Gateway cluster
cluster

In [15]:
if cluster:
    client = Client(cluster) # create a dask Gateway cluster
else:
    client = Client()   # create a local dask cluster on the machine.
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /jupyterhub/services/dask-gateway/clusters/daskhub.b8d8ada755c349c08f4cb8cb36d26f24/status,


In [12]:
#closing open 
clusters=gateway.list_clusters()
print(clusters )
for cluster in clusters :
    cluster= gateway.connect(cluster.name)
    print(cluster)
    client = Client(cluster)
    client.close()
    cluster.shutdown()

[]


In [14]:
# To close at end of session
client.close()

In [ ]:
cluster.shutdown()

## List files

In [5]:
print(s3.ls('escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration')[1])
print(s3.ls('escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration')[-2])

escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20020601-v5.4.nc
escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-20111004-v5.4.nc


In [6]:
print(s3.ls('escience2022/Antoine/AMSR2_ASI_sea_ice_concentration')[1])
print(s3.ls('escience2022/Antoine/AMSR2_ASI_sea_ice_concentration')[-2])

escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20120703-v5.4.nc
escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-20191230-v5.4.nc


## Write Monthly files : AMSR2

In [6]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('/')[-1].split('-')[3], '%Y%m%d') for date in paths]

In [7]:
def read_daily_files(year):
    print('year '+ str(year))
    for month in range (1,13):
        print('month '+ str(month))
        remote_files = 's3://escience2022/Antoine/AMSR2_ASI_sea_ice_concentration/asi-AMSR2-n6250-%i%s*.nc'%(year,str(month).zfill(2))
        remote_files = s3.glob(remote_files)
        fileset = [s3.open(file) for file in remote_files]
        time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))
        try:
            dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
            dset = dset.assign_coords(time=time_var)
            sf = 'data/SIC/AMSR2_ASI_sea_ice_concentration_%i%s.nc'%(year,str(month).zfill(2))
            print('\t saving to '+ sf)
            dset.to_netcdf(sf)
            del remote_files, dset, time_var
        except OSError: 
            del time_var
            pass
        

In [ ]:
with ProgressBar():
    [read_daily_files(year) for year in range(2018,2020)]

## Write Monthly files : AMSR-E

In [9]:
def paths_to_datetimeindex(paths):
    return  [datetime.strptime(date.split('/')[-1].split('-')[2], '%Y%m%d') for date in paths]

In [10]:
def read_daily_files(year):
    print('year '+ str(year))
    for month in range (1,13):
        print('month '+ str(month))
        remote_files = 's3://escience2022/Antoine/AMSR-E_ASI_sea_ice_concentration/asi-n6250-%i%s*.nc'%(year,str(month).zfill(2))
        remote_files = s3.glob(remote_files)
        fileset = [s3.open(file) for file in remote_files]
        time_var = xr.Variable('time', paths_to_datetimeindex(remote_files))
        try:
            dset =xr.open_mfdataset(fileset, concat_dim="time", combine = "nested")
            dset = dset.assign_coords(time=time_var)
            sf = 'data/SIC/AMSR-E_ASI_sea_ice_concentration_%i%s.nc'%(year,str(month).zfill(2))
            print('\t saving to '+ sf)
            dset.to_netcdf(sf)
            del remote_files, dset, time_var
        except OSError: 
            del time_var
            pass
        

In [16]:
[read_daily_files(year) for year in range(2006,2012)]

year 2006
month 1
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200601.nc
month 2
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200602.nc
month 3
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200603.nc
month 4
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200604.nc
month 5
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200605.nc
month 6
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200606.nc
month 7
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200607.nc
month 8
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200608.nc
month 9
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200609.nc
month 10
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200610.nc
month 11
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200611.nc
month 12
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200612.nc
year 2007
month 1
	 saving to data/SIC/AMSR-E_ASI_sea_ice_concentration_200701.nc
month 2
	 saving to data/SIC/AMSR-E_ASI_s

[None, None, None, None, None, None]